In [6]:
import os
import luigi
from emu.pdil.raw import get_data_manifest,Participant
from emu.pipeline.download import FileManifest, ExperimentManifest
from emu.pipeline.remote import Patients
from emu.luigi.box import BoxClient,file_id_to_path,BoxTarget,path_to_fid
from emu.utils import generate_id
import pandas as pd
import numpy as np

### When we collect new data on a patient they need to be added to the patient manifest so the API can find their files

This file is located in the emu root folder named '_patient_manifest.csv'

In [7]:
patients = Patients().load()
patients

,patient_id,md5_16,patient_initials,patient_order,study,folder_id,start,type
0,0,0,OO,0,test,94102978809,'2019-07-30_11-04-51',seeg
1,5783,ae3e7f0a6057f8d3,PC,0,meds,116245295093,NaN,study_root
2,5783,ae3e7f0a6057f8d3,PC,0,meds,116167523390,NaN,seeg
3,595,e2b11b405f0c59a5,CC,0,meds,116245096503,NaN,study_root
4,595,e2b11b405f0c59a5,CC,0,meds,116167113972,NaN,seeg
5,6305,785c1bcf28586305,CH,1,meds,116254528351,NaN,seeg
6,6305,785c1bcf28586305,CH,1,meds,116254201975,NaN,seeg
7,6305,785c1bcf28586305,CH,1,meds,116245336484,NaN,study_root
8,8751,a124696c53387b51,MF,2,meds,116244412027,NaN,study_root
9,6305,785c1bcf28586305,CH,1,pdil,106535506845,D2’,seeg


# New entries

For each new patient they sould at least have a 'study_root' entry which points to that patients root directory on box

If data is collected and stored for that patient we need an entry for each of those data folders and their data type 
> behavioral for behavior data, seeg for seeg data etc.

Excel may represent the folder_id's as floats. This is BAD. Make sure to change the format of cells in the folder_id to a number with no decimal points (so it reperesents them as ints)

In [8]:
generate_id('elijah','christensen')

md5: 4a83b741e9daf43a
patient_id: 1943


'1943'

In [9]:
# pt1_manifest = FileManifest(p)

In [10]:
exp_manifest = ExperimentManifest(study='meds')

In [11]:
exp_manifest.run()

pt_8751: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


Confirm the manifest csv file list was created successfully

In [ ]:
exp_manifest.output().path

In [ ]:
exp_manifest.output().exists()

Load it and make sure it finds the files you'd expect

In [ ]:
exps = pd.read_csv(exp_manifest.output().path)
exps = exps.merge(patients[['patient_id','patient_initials']].drop_duplicates(),on='patient_id')
exps

In [ ]:
exps.groupby(['type','patient_initials','folder'])[['filename']].count()